<a href="https://colab.research.google.com/github/vishal2505/Data_Science/blob/master/HTTP_Logs_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving http_log.txt to http_log.txt


In [22]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [38]:
%%bigquery --project valued-amp-251115
select updated_at, DATETIME(updated_at, "Singapore") as updated_at_SGT from `reward_transaction.reward_transaction`

,updated_at,updated_at_SGT
0,2019-08-18 11:04:10.041000+00:00,2019-08-18 19:04:10.041
1,2019-08-16 11:04:10.041000+00:00,2019-08-16 19:04:10.041
2,2019-08-12 11:04:10.041000+00:00,2019-08-12 19:04:10.041
3,2019-08-13 11:04:10.041000+00:00,2019-08-13 19:04:10.041
4,2019-08-22 11:04:10.041000+00:00,2019-08-22 19:04:10.041
5,2019-08-15 11:04:10.041000+00:00,2019-08-15 19:04:10.041
6,2019-08-14 11:04:10.041000+00:00,2019-08-14 19:04:10.041
7,2019-08-19 11:04:10.041000+00:00,2019-08-19 19:04:10.041
8,2019-08-21 11:04:10.041000+00:00,2019-08-21 19:04:10.041
9,2019-08-17 11:04:10.041000+00:00,2019-08-17 19:04:10.041


In [2]:
import csv
reader = csv.reader(open("http_log.txt"), delimiter=" ")
data = list(reader)
data[0]


['2019-08-27T00:07:05+00:00', 'POST', '/rewards/1529', '2000020076516']

In [3]:
!pip install avro-python3

  Created wheel for avro-python3: filename=avro_python3-1.9.1-cp36-none-any.whl size=43199 sha256=d5f858ff4a319a807c7fc17d6ed125495423682eb51924b152a9e67d24142b65
  Stored in directory: /root/.cache/pip/wheels/94/54/6f/a5df680fd3224aa45145686f3b1b02a878a90ea769fcf9daaf
Successfully built avro-python3


In [5]:
from google.colab import files
uploaded = files.upload()

Saving http_log_schema.avsc to http_log_schema.avsc


In [4]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

schema = avro.schema.Parse(open("http_log_schema.avsc").read())
schema

In [0]:
writer = DataFileWriter(open("http_log.avro", "wb"), DatumWriter(), schema)
for row in data:
  writer.append({"timestamp": row[0], "http_method": row[1], "http_path": row[2], "user_id": row[3]})
writer.close()

In [6]:
avro_reader = DataFileReader(open("http_log.avro", "rb"), DatumReader())
avro_data = list(avro_reader)
print(avro_data[0])
print(avro_data[15999])
avro_reader.close()

{'timestamp': '2019-08-27T00:07:05+00:00', 'http_method': 'POST', 'http_path': '/rewards/1529', 'user_id': '2000020076516'}
{'timestamp': '2019-08-26T16:33:55+00:00', 'http_method': 'GET', 'http_path': '/campaigns/3602', 'user_id': '2000069344719'}


In [7]:
!pip install apache-beam[gcp]

     |████████████████████████████████| 2.9MB 2.8MB/s 
     |████████████████████████████████| 1.2MB 40.1MB/s 
     |████████████████████████████████| 153kB 49.7MB/s 
     |████████████████████████████████| 81kB 26.1MB/s 
     |████████████████████████████████| 51kB 17.8MB/s 
     |████████████████████████████████| 61kB 25.1MB/s 
     |████████████████████████████████| 102kB 31.4MB/s 
     |████████████████████████████████| 174kB 57.3MB/s 
     |████████████████████████████████| 163kB 49.7MB/s 
     |████████████████████████████████| 92kB 31.3MB/s 
     |████████████████████████████████| 112kB 53.2MB/s 
  Created wheel for dill: filename=dill-0.2.9-cp36-none-any.whl size=77403 sha256=4b198e41f14e090b3205a67c8f3b0e8235c348fd96dc2885e05b8d72485fad42
  Stored in directory: /root/.cache/pip/wheels/5b/d7/0f/e58eae695403de585269f4e4a94e0cd6ca60ec0c202936fa4a
  Created wheel for oauth2client: filename=oauth2client-3.0.0-cp36-none-any.whl size=106382 sha256=cdbd7678bf35de745128573c1531ed106961

In [55]:
from google_auth_oauthlib import flow

# TODO: Uncomment the line below to set the `launch_browser` variable.
# launch_browser = True
#
# The `launch_browser` boolean variable indicates if a local server is used
# as the callback URL in the auth flow. A value of `True` is recommended,
# but a local server does not work if accessing the application remotely,
# such as over SSH or from a remote Jupyter notebook.

appflow = flow.InstalledAppFlow.from_client_secrets_file(
    'client_secrets.json',
    scopes=['https://www.googleapis.com/auth/bigquery'])

appflow.run_console()

credentials = appflow.credentials

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=162956633472-9ek54hnj38949gjsv8u450enu91hddot.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=YDGnpkmA8LRv3IpHGkzIDwfXv0bZQX&prompt=consent&access_type=offline&code_challenge=qemTUOVdgCIBSeBmKvNNjy3Ttb_U77JeBMp5iJNNqDE&code_challenge_method=S256
Enter the authorization code: 4/qgG3m0vqz-eL6q3mBKanSdD9inVvxY-huZhB4dMuKSbzNsIzlVyilV4


In [56]:
from apache_beam.io.gcp.internal.clients import bigquery
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions

os.environ['GOOGLE_APPLICATION_CREDENTIALS']="client_secrets.json"

CLIENT_ID = "162956633472-9ek54hnj38949gjsv8u450enu91hddot.apps.googleusercontent.com"
os.environ['client_secret'] = "mceD7sGjhWd2ojvn8bWf5RJB"

table_spec = bigquery.TableReference(
    projectId='http-log-analyser',
    datasetId='http_logs',
    tableId='http_logs')

p = beam.Pipeline(options=PipelineOptions())

table_schema = ('timestamp:STRING, http_method:STRING, http_path:STRING, user_id:STRING')

data = (p | beam.io.ReadFromAvro('http_log.avro'))

data | beam.io.WriteToBigQuery(
table_spec,
schema=table_schema,
create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE)

p.run().wait_until_finish()

/usr/local/lib/python3.6/dist-packages/apache_beam/io/gcp/bigquery.py:1150: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []


RuntimeError: ignored